In [30]:
# -*- coding: utf-8 -*-
import requests
import json
import re
import time

from datetime import datetime
import pandas as pd
import numpy as np

# from bs4 import BeautifulSoup
# from selenium import webdriver
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.common.desired_capabilities import DesiredCapabilities  
# from selenium.webdriver.support import expected_conditions as EC
# from dianping import get_dianping

class StopSpyder(BaseException):pass

class NewsGain(object):

    # 初始化
    def __init__(self):
        self.login = False
        self.data = pd.DataFrame(columns = ["source","spot","time","userid","content","star"])

    # 获取BeautifulSoup对象
    def getBS(self,url,params = None):
        #定义访问文件头
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'}
        # 访问网页
        req = requests.get(url, headers = headers, params=params)
        # 获取编码
        charset = req.encoding
        # 获取网页内容
        content = req.text
        # 先根据原编码转化为字节bytes，转化为utf-8字符串str
        content = content.encode(charset).decode('utf-8')
        # 获取bs对象，类型未lxmlfin
        soup = BeautifulSoup(content, 'lxml')

#        soup = BeautifulSoup(content, 'html.parser')
        # 去掉style标签
        [s.extract() for s in soup('style')]
        # 返回soup
        return soup

    # 保存数据到data
    def saveData(self, source, spot, time, userid, content, star):

        index = self.data.shape[0]
        self.data.at[index,'source'] = source
        self.data.at[index,'spot'] = spot
        self.data.at[index,'time'] = time
        self.data.at[index,'userid'] = userid
        self.data.at[index,'content'] = content
        self.data.at[index,'star'] = star

    # Test 导出csv文件
    def toCSV(self,name, charset = 'utf-8'):
        try:
            self.data.to_csv('{}.csv'.format(name), index = False, encoding = charset)
        except Exception as e:
            pass

    #新闻爬取入口
    def startGain(self,url):
        if url[12] == 'm':
            self.meituan(url = url)
        elif url[7:9] == 'ti':
            self.lvmama(url = url)
        elif url[13:18] == 'qunar':
            self.qunar(url = url)
        else:
            pass


    # 美团网        
    def meituan(self, url):
        # 美团评价采用动态网页，访问对应的接口解析JSON即可
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'}

        # 定义源
        source = u'美团'

        # 访问网页
        req = requests.get(url,headers = headers)

        # 获取内容
        charset = req.encoding
        content = req.text.encode(charset).decode('utf-8')

        # 转化为JSON对象
        json_con = json.loads(content)

        # 获取评论列表
        comment_list = json_con['comments']
        if not comment_list:
            return ''

        # 根据url中的poiId判断地点
        if url[48:55] == '1571217':
            # 地点
            spot = '周庄'
        else:
            spot = '乌镇'

        # 获取具体内容
        for comment in comment_list:
            # 评论时间
            com_time = datetime.fromtimestamp(int(comment['commentTime'])/1000).strftime('%Y-%m-%d %H:%M:%S')
            # 用户
            userid = comment['userName']
            # 内容
            content = str(comment['comment'])
            # 星级
            star = comment['star']/10
            self.saveData(source,spot,com_time,userid,content,star)

    # 去哪儿    
    def qunar(self, url):
        # 网站来源
        source = u'去哪儿'

        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'}

        # 根据url中的poiId判断地点
        if url[72:76] == '5336':
            # 地点
            spot = '周庄'
        elif url[72:77] == '13231':
            spot = '乌镇'
        else:
            spot = ''

        # 访问网页
        req = requests.get(url,headers = headers)

        # 获取内容
        charset = req.encoding
        content = req.text.encode(charset).decode('utf-8')

        # 转化为JSON对象
        json_con = json.loads(content)

        # 获取评论列表
        comment_list = json_con['data']['commentList']

        # 获取具体内容
        for comment in comment_list:
            userid = comment['author']
            com_time = comment['date']
            star = comment['score']
            content = str(comment['content'])
            self.saveData(source,spot,com_time,userid,content,star)

    # 驴妈妈
    def lvmama(self, url):

        # 获取参数信息
        lvDir = {'http://ticket.lvmama.com/scenic-103108':{'currentPage':9,'placeId':'103108','spot':'大伊山'},
                 'http://ticket.lvmama.com/scenic-11345447':{'currentPage':None,'placeId':'11345447','spot':'伊甸园'},
                 'http://ticket.lvmama.com/scenic-11345379':{'currentPage':None,'placeId':'11345379','spot':'伊芦山'}
                 }
        if lvDir[url]['currentPage'] is None:
            return 

        # 网站来源
        source = u'驴妈妈'
        spot = lvDir[url]['spot']
        # 驴妈妈POSTurl
        rurl = 'http://ticket.lvmama.com/vst_front/comment/newPaginationOfComments'


        # 获取参数
        for i in range(1,lvDir[url]['currentPage']):
            print(i)

            # 配置参数
            params = {
            'currentPage':i,
            'isBest':None,
            'isELong':'N',
            'isPicture':None,
            'isPOI':'Y',
            'placeId':lvDir[url]['placeId'],
            'placeIdType':'PLACE',
            'productId':None,
            'totalCount':72,
            'type':'all'
            }

            # 获取soup对象

            soup = self.getBS(rurl,params)
            # 获取评论列表
            comment_list = soup.find_all(class_ = 'comment-li')
            # 遍历新闻，获取每条的详细信息
            for comment in comment_list:

                star = comment.find(class_ = 'ufeed-level').i.get('data-level')
                userid = comment.find(class_ = 'com-userinfo').p.a.get('title')
                time = comment.find(class_ = 'com-userinfo').p.em.text
                content = comment.find(class_ = 'ufeed-content').text.replace('\r\n','').replace(' ','')
#                print(content)

                if time > self.get_last_time(source,spot):
                    self.saveData(source,spot,time,userid,content,star)
                 

# 程序入口
if __name__ == "__main__":
    # 目标爬取网站
#                  'http://ticket.lvmama.com/scenic-103108', #驴妈妈大伊山
#                  'http://ticket.lvmama.com/scenic-11345447', #驴妈妈伊甸园
#                  'http://ticket.lvmama.com/scenic-11345379', #驴妈妈伊芦山梅园

    
    mt_url = ['https://www.meituan.com/ptapi/poi/getcomment?id={}&offset={}&pageSize=200&sortType=1'.format(j[0],i*100) for j in [(1571217,100),(711643,10),(1192715,52),(52427083,44)] for i in range(0,j[1],2) ]
    quna_url = ['https://piao.qunar.com/ticket/detailLight/sightCommentList.json?sightId=5336&index={}&page={}&pageSize=100&tagType=0'.format(i,i) for i in range(1,136)]
    quna_url += ['https://piao.qunar.com/ticket/detailLight/sightCommentList.json?sightId=13231&index={}&page={}&pageSize=100&tagType=0'.format(i,i) for i in range(1,164)]
    """
    https://www.meituan.com/ptapi/poi/getcomment?id=1571217&offset={}&pageSize=200&sortType=1  # 周庄 9118
    https://www.meituan.com/ptapi/poi/getcomment?id=711643&offset=10&pageSize=10&mode=0&starRange=&userId=&sortType=1 # 乌镇 856
    https://www.meituan.com/ptapi/poi/getcomment?id=1192715&offset=10&pageSize=10&mode=0&starRange=&userId=&sortType=1  西 5147
    https://www.meituan.com/ptapi/poi/getcomment?id=52427083&offset=10&pageSize=10&mode=0&starRange=&userId=&sortType=1  东4391
    https://piao.qunar.com/ticket/detailLight/sightCommentList.json?sightId=13231&index=164&page=2&pageSize=100&tagType=0
    """
    # 获取爬取程序实例
    newsGain = NewsGain()

    # 爬取数据
    for url in mt_url:
        print(url)
        newsGain.startGain(url)
    newsGain.toCSV('mt')
    for url in quna_url:
        print(url)
        newsGain.startGain(url)
    newsGain.toCSV('total')


https://www.meituan.com/ptapi/poi/getcomment?id=1571217&offset=0&pageSize=200&sortType=1
https://www.meituan.com/ptapi/poi/getcomment?id=1571217&offset=200&pageSize=200&sortType=1
https://www.meituan.com/ptapi/poi/getcomment?id=1571217&offset=400&pageSize=200&sortType=1
https://www.meituan.com/ptapi/poi/getcomment?id=1571217&offset=600&pageSize=200&sortType=1
https://www.meituan.com/ptapi/poi/getcomment?id=1571217&offset=800&pageSize=200&sortType=1
https://www.meituan.com/ptapi/poi/getcomment?id=1571217&offset=1000&pageSize=200&sortType=1
https://www.meituan.com/ptapi/poi/getcomment?id=1571217&offset=1200&pageSize=200&sortType=1
https://www.meituan.com/ptapi/poi/getcomment?id=1571217&offset=1400&pageSize=200&sortType=1
https://www.meituan.com/ptapi/poi/getcomment?id=1571217&offset=1600&pageSize=200&sortType=1
https://www.meituan.com/ptapi/poi/getcomment?id=1571217&offset=1800&pageSize=200&sortType=1
https://www.meituan.com/ptapi/poi/getcomment?id=1571217&offset=2000&pageSize=200&sortTy

https://www.meituan.com/ptapi/poi/getcomment?id=52427083&offset=1800&pageSize=200&sortType=1
https://www.meituan.com/ptapi/poi/getcomment?id=52427083&offset=2000&pageSize=200&sortType=1
https://www.meituan.com/ptapi/poi/getcomment?id=52427083&offset=2200&pageSize=200&sortType=1
https://www.meituan.com/ptapi/poi/getcomment?id=52427083&offset=2400&pageSize=200&sortType=1
https://www.meituan.com/ptapi/poi/getcomment?id=52427083&offset=2600&pageSize=200&sortType=1
https://www.meituan.com/ptapi/poi/getcomment?id=52427083&offset=2800&pageSize=200&sortType=1
https://www.meituan.com/ptapi/poi/getcomment?id=52427083&offset=3000&pageSize=200&sortType=1
https://www.meituan.com/ptapi/poi/getcomment?id=52427083&offset=3200&pageSize=200&sortType=1
https://www.meituan.com/ptapi/poi/getcomment?id=52427083&offset=3400&pageSize=200&sortType=1
https://www.meituan.com/ptapi/poi/getcomment?id=52427083&offset=3600&pageSize=200&sortType=1
https://www.meituan.com/ptapi/poi/getcomment?id=52427083&offset=3800&p

https://piao.qunar.com/ticket/detailLight/sightCommentList.json?sightId=5336&index=61&page=61&pageSize=100&tagType=0
https://piao.qunar.com/ticket/detailLight/sightCommentList.json?sightId=5336&index=62&page=62&pageSize=100&tagType=0
https://piao.qunar.com/ticket/detailLight/sightCommentList.json?sightId=5336&index=63&page=63&pageSize=100&tagType=0
https://piao.qunar.com/ticket/detailLight/sightCommentList.json?sightId=5336&index=64&page=64&pageSize=100&tagType=0
https://piao.qunar.com/ticket/detailLight/sightCommentList.json?sightId=5336&index=65&page=65&pageSize=100&tagType=0
https://piao.qunar.com/ticket/detailLight/sightCommentList.json?sightId=5336&index=66&page=66&pageSize=100&tagType=0
https://piao.qunar.com/ticket/detailLight/sightCommentList.json?sightId=5336&index=67&page=67&pageSize=100&tagType=0
https://piao.qunar.com/ticket/detailLight/sightCommentList.json?sightId=5336&index=68&page=68&pageSize=100&tagType=0
https://piao.qunar.com/ticket/detailLight/sightCommentList.json?

https://piao.qunar.com/ticket/detailLight/sightCommentList.json?sightId=5336&index=131&page=131&pageSize=100&tagType=0
https://piao.qunar.com/ticket/detailLight/sightCommentList.json?sightId=5336&index=132&page=132&pageSize=100&tagType=0
https://piao.qunar.com/ticket/detailLight/sightCommentList.json?sightId=5336&index=133&page=133&pageSize=100&tagType=0
https://piao.qunar.com/ticket/detailLight/sightCommentList.json?sightId=5336&index=134&page=134&pageSize=100&tagType=0
https://piao.qunar.com/ticket/detailLight/sightCommentList.json?sightId=5336&index=135&page=135&pageSize=100&tagType=0
https://piao.qunar.com/ticket/detailLight/sightCommentList.json?sightId=13231&index=1&page=1&pageSize=100&tagType=0
https://piao.qunar.com/ticket/detailLight/sightCommentList.json?sightId=13231&index=2&page=2&pageSize=100&tagType=0
https://piao.qunar.com/ticket/detailLight/sightCommentList.json?sightId=13231&index=3&page=3&pageSize=100&tagType=0
https://piao.qunar.com/ticket/detailLight/sightCommentLis

https://piao.qunar.com/ticket/detailLight/sightCommentList.json?sightId=13231&index=66&page=66&pageSize=100&tagType=0
https://piao.qunar.com/ticket/detailLight/sightCommentList.json?sightId=13231&index=67&page=67&pageSize=100&tagType=0
https://piao.qunar.com/ticket/detailLight/sightCommentList.json?sightId=13231&index=68&page=68&pageSize=100&tagType=0
https://piao.qunar.com/ticket/detailLight/sightCommentList.json?sightId=13231&index=69&page=69&pageSize=100&tagType=0
https://piao.qunar.com/ticket/detailLight/sightCommentList.json?sightId=13231&index=70&page=70&pageSize=100&tagType=0
https://piao.qunar.com/ticket/detailLight/sightCommentList.json?sightId=13231&index=71&page=71&pageSize=100&tagType=0
https://piao.qunar.com/ticket/detailLight/sightCommentList.json?sightId=13231&index=72&page=72&pageSize=100&tagType=0
https://piao.qunar.com/ticket/detailLight/sightCommentList.json?sightId=13231&index=73&page=73&pageSize=100&tagType=0
https://piao.qunar.com/ticket/detailLight/sightCommentLi

https://piao.qunar.com/ticket/detailLight/sightCommentList.json?sightId=13231&index=135&page=135&pageSize=100&tagType=0
https://piao.qunar.com/ticket/detailLight/sightCommentList.json?sightId=13231&index=136&page=136&pageSize=100&tagType=0
https://piao.qunar.com/ticket/detailLight/sightCommentList.json?sightId=13231&index=137&page=137&pageSize=100&tagType=0
https://piao.qunar.com/ticket/detailLight/sightCommentList.json?sightId=13231&index=138&page=138&pageSize=100&tagType=0
https://piao.qunar.com/ticket/detailLight/sightCommentList.json?sightId=13231&index=139&page=139&pageSize=100&tagType=0
https://piao.qunar.com/ticket/detailLight/sightCommentList.json?sightId=13231&index=140&page=140&pageSize=100&tagType=0
https://piao.qunar.com/ticket/detailLight/sightCommentList.json?sightId=13231&index=141&page=141&pageSize=100&tagType=0
https://piao.qunar.com/ticket/detailLight/sightCommentList.json?sightId=13231&index=142&page=142&pageSize=100&tagType=0
https://piao.qunar.com/ticket/detailLigh

In [29]:
# ['https://www.meituan.com/ptapi/poi/getcomment?id={}&offset={}&pageSize=200&sortType=1'.format(j[0],i*100) for j in [(1571217,100),(711643,10),(1192715,52),(52427083,44)] for i in range(0,j[1],2) ]
'https://piao.qunar.com/ticket/detailLight/sightCommentList.json?sightId=13231&index=164&page=2&pageSize=100&tagType=0'[72:76]

'1323'

In [10]:
from datetime import datetime
datetime.fromtimestamp(1550115052).strftime('%Y-%m-%d %H:%M:%S')

'2019-02-14 11:30:52'